In [3]:
from datasets import load_from_disk

dataset = load_from_disk("data/processed/train")
print(dataset)



Dataset({
    features: ['domain', 'title', 'doc', 'entities', 'triples', 'label_set', 'entity_label_set', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 51
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

import numpy as np

id_to_label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-MISC",
    8: "I-MISC"
}

def predict_example(example):
    inputs = tokenizer(example["doc"], truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_ids = outputs.logits.argmax(dim=-1).squeeze().tolist()
    
    predicted_labels = [id_to_label.get(id_, "O") for id_ in predicted_ids]
    
    return predicted_labels

predictions = []
for example in val_dataset:
    preds = predict_example(example)
    predictions.append(preds)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/daniellocher/micromamba/envs/nlp_project/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/daniellocher/micromamba/envs/nlp_project/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/daniellocher/micromamba/envs/nlp_project/lib/python3.12/site-packages/ipykernel/ker

NameError: name 'val_dataset' is not defined

In [ ]:
def convert_true_labels(label_ids, id_to_label):
    return [id_to_label.get(label, "O") for label in label_ids if label != -100]

true_labels = []
for example in val_dataset:
    true = convert_true_labels(example["labels"], id_to_label)
    true_labels.append(true)


In [ ]:
from seqeval.metrics import classification_report

report = classification_report(true_labels, predictions)
print(report)
